### Import Required Libraries and Set Up Environment Variables

In [158]:
#I did the best I could. There were many errors in the starting code and error and incosistancies with the api docuemntation that stopped me from succeeding on this assignment.
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [165]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [160]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:("Movies")&type_of_material:("Review")&headline:("love")'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
#IMPORTANT: THIS SEEMS TO BE BROKEN ON THE WEBSITE. OMMITING
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url = url + "api-key=" + nyt_api_key + "&q=" + filter_query + "&sort=" + sort + "&fl=" + field_list# + "&begin_date=" + begin_date# + "&end_date=" + end_date

In [161]:
# Create an empty list to store the reviews
reviews_list = []
# loop through pages 0-19
for i in range(0,20):
    # create query with a page number
    # API results show 10 articles at a time
    query_url_this = query_url + f"&page={i}"
    # Make a "GET" request and retrieve the JSON
    response = requests.get(query_url)
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    # Try and save the reviews to the reviews_list
    if response.status_code == 200:
        response_json = response.json()
        # loop through the reviews["response"]["docs"] and append each review to the list
        reviews_list.extend(response_json["response"]["docs"])
        # Print the page that was just retrieved
        print(f"Retrieved page {i} with code {response.status_code}.")
        # Print the page number that had no results then break from the loop
    else:
        print(f"Error: Failed to fetch data for page {i}")
        break

KeyboardInterrupt: 

In [ ]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data

preview_data = reviews_list[:5]
print(json.dumps(preview_data, indent=4))

[
    {
        "web_url": "https://www.nytimes.com/1980/08/29/archives/at-the-movies-breaking-through-in-airplane.html",
        "snippet": "",
        "source": "The New York Times",
        "headline": {
            "main": "At the Movies; Breaking through in 'Airplane!'",
            "kicker": "1",
            "content_kicker": null,
            "print_headline": "At the Movies; Breaking through in 'Airplane!'",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [],
        "pub_date": "1980-08-29T05:00:00+0000",
        "byline": {
            "original": "Judy Klemesrud",
            "person": [
                {
                    "firstname": "Judy",
                    "middlename": null,
                    "lastname": "Klemesrud",
                    "qualifier": null,
                    "title": null,
                    "role": "reported",
                    "organization": "",
                    "rank": 1
        

In [ ]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
nyt_reviews = pd.json_normalize(reviews_list)
nyt_reviews_df = pd.DataFrame(nyt_reviews)
nyt_reviews_df_backup =nyt_reviews.copy
nyt_reviews_df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/1980/08/29/archives/at...,,The New York Times,[],1980-08-29T05:00:00+0000,0,At the Movies; Breaking through in 'Airplane!',1,None,At the Movies; Breaking through in 'Airplane!',None,None,None,Judy Klemesrud,"[{'firstname': 'Judy', 'middlename': None, 'la...",None
1,https://www.nytimes.com/1980/08/22/archives/at...,,The New York Times,[],1980-08-22T05:00:00+0000,0,At the Movies; Ray Sharkey talks about Phil an...,1,None,At the Movies; Ray Sharkey talks about Phil an...,None,None,None,Judy Klemesrud,"[{'firstname': 'Judy', 'middlename': None, 'la...",None
2,https://www.nytimes.com/1980/08/10/archives/le...,,The New York Times,[],1980-08-10T05:00:00+0000,0,Letters to the Editor; Hut-to-Hut Hiking Footn...,1,None,Letters to the Editor; Hut-to-Hut Hiking Footn...,None,None,None,KENNETH N. WHITEFLORENCE M. SHAPIROLEW SHOSKES...,"[{'firstname': 'Kenneth', 'middlename': 'N.', ...",None
3,https://www.nytimes.com/1980/07/20/archives/le...,,The New York Times,[],1980-07-20T05:00:00+0000,0,Letters to the Editor; Personal Safaris Movies...,1,None,Letters to the Editor; Personal Safaris Movies...,None,None,None,MARGARET DRURYMRS. EDWARD PLOTTLELILLIAN ORROK...,"[{'firstname': 'Margaret', 'middlename': None,...",None
4,https://www.nytimes.com/1980/06/08/archives/le...,,The New York Times,[],1980-06-08T05:00:00+0000,0,LETTERS; Inflated Profits Foreign Cars The Mov...,1,None,LETTERS; Inflated Profits Foreign Cars The Mov...,None,None,None,HAROLD M. WILLIAMSROBERT AMSTERTHOMAS G. MORGA...,"[{'firstname': 'Harold', 'middlename': None, '...",None


In [ ]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
def extract_title(headline):
    # Check if the headline contains single quotes
    if "'" in headline:
        # Extract text between single quotes
        start_index = headline.find("'")
        end_index = headline.find("'", start_index + 1)
        if end_index != -1:
            return headline[start_index + 1:end_index]
    # Return the same value if no single quotes found or extraction failed
    return headline

nyt_reviews_df['title'] = nyt_reviews_df['headline.main'].apply(extract_title)
nyt_reviews_df.tail()



,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
195,https://www.nytimes.com/1980/03/09/archives/fi...,,The New York Times,[],1980-03-09T05:00:00+0000,0,FILM VIEW; The Film Fan's Book Guide FILM VIEW,2,None,FILM VIEW; The Film Fan's Book Guide FILM VIEW,None,None,None,VINCENT CANBY,"[{'firstname': 'Vincent', 'middlename': None, ...",None,FILM VIEW; The Film Fan's Book Guide FILM VIEW
196,https://www.nytimes.com/1967/06/23/archives/us...,New Rochelle residents give Fed authorities se...,The New York Times,"[{'name': 'glocations', 'value': 'New York Sta...",1967-06-23T05:00:00+0000,0,U.S. Given Movies on Gambling in New Rochelle;...,None,None,U.S. Given Movies on Gambling in New Rochelle;...,None,None,None,By Martin Arnold Special To the New York Times,"[{'firstname': 'Martin', 'middlename': 'Arnold...",None,U.S. Given Movies on Gambling in New Rochelle;...
197,https://www.nytimes.com/1967/01/01/archives/si...,,The New York Times,[],1967-01-01T05:00:00+0000,0,'Sister George' Preens for the Movies; More Ab...,2,None,'Sister George' Preens for the Movies; More Ab...,None,None,None,By A.h. Weiler,"[{'firstname': None, 'middlename': None, 'last...",None,Sister George
198,https://www.nytimes.com/1966/09/04/archives/mo...,,The New York Times,[],1966-09-04T05:00:00+0000,0,Movies; Dancing 'Up the Down Staircase',1,None,Movies; Dancing 'Up the Down Staircase',None,None,None,By Ronald Gold,"[{'firstname': 'Ronald', 'middlename': None, '...",None,Up the Down Staircase
199,https://www.nytimes.com/1965/10/17/archives/th...,,The New York Times,[],1965-10-17T05:00:00+0000,0,This Spy Is a Cockney,1,None,This Spy Is a Cockney,None,None,None,By Howard Thompson,"[{'firstname': 'Howard', 'middlename': None, '...",None,This Spy Is a Cockney


In [ ]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
nyt_reviews_df['keywords'] = nyt_reviews_df['keywords'].apply(extract_keywords)
nyt_reviews_df.head


<bound method NDFrame.head of                                                web_url  \
0    https://www.nytimes.com/1980/08/29/archives/at...   
1    https://www.nytimes.com/1980/08/22/archives/at...   
2    https://www.nytimes.com/1980/08/10/archives/le...   
3    https://www.nytimes.com/1980/07/20/archives/le...   
4    https://www.nytimes.com/1980/06/08/archives/le...   
..                                                 ...   
195  https://www.nytimes.com/1980/03/09/archives/fi...   
196  https://www.nytimes.com/1967/06/23/archives/us...   
197  https://www.nytimes.com/1967/01/01/archives/si...   
198  https://www.nytimes.com/1966/09/04/archives/mo...   
199  https://www.nytimes.com/1965/10/17/archives/th...   

                                               snippet              source  \
0                                                       The New York Times   
1                                                       The New York Times   
2                                      

In [ ]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles = nyt_reviews_df['title'].to_list()
print(titles)


['Airplane!', 'At the Movies; Ray Sharkey talks about Phil and Fame. ', 'Letters to the Editor; Hut-to-Hut Hiking Footnote to History The Lippizaner War Rubber Trees More on Movies Aloft ', 'Hold', 'LETTERS; Inflated Profits Foreign Cars The Movies Energy Savers Anderson ', "FILM VIEW; The Film Fan's Book Guide FILM VIEW ", 'U.S. Given Movies on Gambling in New Rochelle; Residents Led by a Minister Also Obtained Tapes on Undercover Activities', 'Sister George', 'Up the Down Staircase', 'This Spy Is a Cockney', 'Airplane!', 'At the Movies; Ray Sharkey talks about Phil and Fame. ', 'Letters to the Editor; Hut-to-Hut Hiking Footnote to History The Lippizaner War Rubber Trees More on Movies Aloft ', 'Hold', 'LETTERS; Inflated Profits Foreign Cars The Movies Energy Savers Anderson ', "FILM VIEW; The Film Fan's Book Guide FILM VIEW ", 'U.S. Given Movies on Gambling in New Rochelle; Residents Led by a Minister Also Obtained Tapes on Undercover Activities', 'Sister George', 'Up the Down Stairc

### Access The Movie Database API

In [180]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key
print(tmdb_key_string)

&api_key=fa28d866cc8a3d6b0241fead3b8ba0b1


In [184]:
# Create an empty list to store the results
tmdb_reviews_list = []
genre_list = []
id_list =[]
lang_list =[]
prod_countries=[]


# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 0
    

# Loop through the titles
for title in titles:
    print(url + title + tmdb_key_string)
    # Check if we need to sleep before making a request
    if request_counter % 50 == 0 and request_counter != 0:
        time.sleep(10) 

    # Add 1 to the request counter
    request_counter += 1

    
    # Perform a "GET" request for The Movie Database
    response = requests.get(url + title + tmdb_key_string)
    

    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:
        # Get movie id
        print(json.dumps(response.json(), indent=4))
        movie_id = response.json()["results"][0]["id"]
        id_list.append(movie_id)
        print(f"Movie id for {title} is {movie_id}")
        # Make a request for a the full movie details
        #movie_details_url = f"https://api.themoviedb.org/3/movie/{movie_id}"
        # Execute "GET" request with url
        #My key will not work for the request.
        #movie_response = requests.get(movie_details_url + tmdb_key_string).json()
        # Extract the genre names into a list
        genre_list.append(response.json()["results"][0]["genre_ids"])
        # Extract the spoken_languages' English name into a list
        lang_list.append(response.json()["results"][0]["original_language"])
        # Extract the production_countries' name into a list
        #prod_countries.extend(response.json()["results"][0]["production_countries"])
        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        tmdb_reviews_list.append(response.json()["results"][0])
        
        # Print out the title that was found
        print(response.json()["results"][0]["id"])
        time.sleep(1)
    except IndexError:
        print(f"Movie not found for title: {title}")

https://api.themoviedb.org/3/search/movie?query=Airplane!&api_key=fa28d866cc8a3d6b0241fead3b8ba0b1
{
    "page": 1,
    "results": [
        {
            "adult": false,
            "backdrop_path": "/qOS1GpVfRxSqHmtUglt8yisBc4v.jpg",
            "genre_ids": [
                35
            ],
            "id": 813,
            "original_language": "en",
            "original_title": "Airplane!",
            "overview": "An ex-fighter pilot forced to take over the controls of an airliner when the flight crew succumbs to food poisoning.",
            "popularity": 55.829,
            "poster_path": "/7Q3efxd3AF1vQjlSxnlerSA7RzN.jpg",
            "release_date": "1980-07-02",
            "title": "Airplane!",
            "video": false,
            "vote_average": 7.299,
            "vote_count": 4266
        },
        {
            "adult": false,
            "backdrop_path": "/9OaIWmWI7Ph3jqj235zQIrh5yVd.jpg",
            "genre_ids": [
                35,
                18,
      

KeyError: 'results'

In [185]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
preview_data = tmdb_reviews_list[:5]
print(json.dumps(preview_data, indent=4))

[
    {
        "adult": false,
        "backdrop_path": "/qOS1GpVfRxSqHmtUglt8yisBc4v.jpg",
        "genre_ids": [
            35
        ],
        "id": 813,
        "original_language": "en",
        "original_title": "Airplane!",
        "overview": "An ex-fighter pilot forced to take over the controls of an airliner when the flight crew succumbs to food poisoning.",
        "popularity": 55.829,
        "poster_path": "/7Q3efxd3AF1vQjlSxnlerSA7RzN.jpg",
        "release_date": "1980-07-02",
        "title": "Airplane!",
        "video": false,
        "vote_average": 7.299,
        "vote_count": 4266
    }
]


In [194]:
# Convert the results to a DataFrame
tmdb_reviews = pd.json_normalize(tmdb_reviews_list)
tmdb_reviews_df = pd.DataFrame(tmdb_reviews)
tmdb_reviews_df_backup =tmdb_reviews_df.copy
tmdb_reviews_df.head()

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
0,False,/qOS1GpVfRxSqHmtUglt8yisBc4v.jpg,[35],813,en,Airplane!,An ex-fighter pilot forced to take over the co...,55.829,/7Q3efxd3AF1vQjlSxnlerSA7RzN.jpg,1980-07-02,Airplane!,False,7.299,4266


### Merge and Clean the Data for Export

In [195]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(nyt_reviews_df, tmdb_reviews_df, on="title", how="inner")
merged_df.set_index('title')
merged_df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,id,original_language,original_title,overview,popularity,poster_path,release_date,video,vote_average,vote_count
0,https://www.nytimes.com/1980/08/29/archives/at...,,The New York Times,,1980-08-29T05:00:00+0000,0,At the Movies; Breaking through in 'Airplane!',1,None,At the Movies; Breaking through in 'Airplane!',...,813,en,Airplane!,An ex-fighter pilot forced to take over the co...,55.829,/7Q3efxd3AF1vQjlSxnlerSA7RzN.jpg,1980-07-02,False,7.299,4266
1,https://www.nytimes.com/1980/08/29/archives/at...,,The New York Times,,1980-08-29T05:00:00+0000,0,At the Movies; Breaking through in 'Airplane!',1,None,At the Movies; Breaking through in 'Airplane!',...,813,en,Airplane!,An ex-fighter pilot forced to take over the co...,55.829,/7Q3efxd3AF1vQjlSxnlerSA7RzN.jpg,1980-07-02,False,7.299,4266
2,https://www.nytimes.com/1980/08/29/archives/at...,,The New York Times,,1980-08-29T05:00:00+0000,0,At the Movies; Breaking through in 'Airplane!',1,None,At the Movies; Breaking through in 'Airplane!',...,813,en,Airplane!,An ex-fighter pilot forced to take over the co...,55.829,/7Q3efxd3AF1vQjlSxnlerSA7RzN.jpg,1980-07-02,False,7.299,4266
3,https://www.nytimes.com/1980/08/29/archives/at...,,The New York Times,,1980-08-29T05:00:00+0000,0,At the Movies; Breaking through in 'Airplane!',1,None,At the Movies; Breaking through in 'Airplane!',...,813,en,Airplane!,An ex-fighter pilot forced to take over the co...,55.829,/7Q3efxd3AF1vQjlSxnlerSA7RzN.jpg,1980-07-02,False,7.299,4266
4,https://www.nytimes.com/1980/08/29/archives/at...,,The New York Times,,1980-08-29T05:00:00+0000,0,At the Movies; Breaking through in 'Airplane!',1,None,At the Movies; Breaking through in 'Airplane!',...,813,en,Airplane!,An ex-fighter pilot forced to take over the co...,55.829,/7Q3efxd3AF1vQjlSxnlerSA7RzN.jpg,1980-07-02,False,7.299,4266


In [196]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing


# Create a list of characters to remove


# Loop through the list of columns to fix

    # Convert the column to type 'str'


    # Loop through characters to remove


# Display the fixed DataFrame


In [197]:
# Drop "byline.person" column
merged_df.drop(columns=['byline.person'], inplace=True)

In [198]:
# Delete duplicate rows and reset index


In [199]:
# Export data to CSV without the index
merged_df.to_csv('movie_info.csv')